# Create Grid Stores
Since the grid information is static across the datasets, we create zarr stores for each grid including:
* Ice (ice)
* Land (lnd)
* Atmosphere (atm)
* Ocean (ocn)

## Imports

In [36]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import zarr
import intake

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup a few helper functions

In [2]:
def create_grid_dataset(sample_file, variables, vars_to_drop=None):
    ds = xr.open_dataset(sample_file, decode_times=False)
    grid_vars = get_grid_vars(ds, variables)
    grid = ds.set_coords(grid_vars)[grid_vars]
    if vars_to_drop:
        to_drop = {vname for vname in vars_to_drop if vname in grid.coords}
        grid = grid.drop(list(to_drop))
    grid.attrs = {}
    return grid

In [3]:
def get_grid_vars(ds, variables):
    coord_vars = [
        vname
        for vname in ds.data_vars
        if 'time' not in ds[vname].dims or 'bound' in vname
    ]
    ds_fixed = ds.set_coords(coord_vars)
    grid_vars = list(ds_fixed.drop_dims('time').coords.keys())
    return grid_vars

## Read in the intake-esm catalog to gather file paths from

In [38]:
col = intake.open_esm_datastore(
    "/glade/work/mgrover/intake-esm-catalogs/cesm2-le.json",
)
col

/glade/u/home/mgrover/miniconda3/envs/cesm2-marbl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,unique
component,6
stream,25
case,36
member_id,20
experiment,2
forcing,2
experiment_number,14
variable,1868
start_time,184
end_time,185


## Create the Grid Zarr Stores

### Sea ice component

In [39]:
# experiment number
exp_num = 1021

# component
comp = 'ice'

# variable
var = 'hi_d'

# variable for this dataset
path = col.search(component=comp, variable=var, experiment_number=exp_num).df.path.values[0]

In [43]:
ice_grid = create_grid_dataset(path, variables=[var])
ice_grid

<xarray.Dataset>
Dimensions:      (nc: 5, ni: 320, nj: 384, nkbio: 5, nkice: 8, nksnow: 3, nvertices: 4)
Coordinates: (12/23)
    TLON         (nj, ni) float32 ...
    TLAT         (nj, ni) float32 ...
    ULON         (nj, ni) float32 ...
    ULAT         (nj, ni) float32 ...
    NCAT         (nc) float32 ...
    VGRDi        (nkice) float32 ...
    ...           ...
    ANGLE        (nj, ni) float32 ...
    ANGLET       (nj, ni) float32 ...
    lont_bounds  (nj, ni, nvertices) float32 ...
    latt_bounds  (nj, ni, nvertices) float32 ...
    lonu_bounds  (nj, ni, nvertices) float32 ...
    latu_bounds  (nj, ni, nvertices) float32 ...
Dimensions without coordinates: nc, ni, nj, nkbio, nkice, nksnow, nvertices
Data variables:
    *empty*

In [42]:
ice_grid.to_zarr(
    "/glade/scratch/mgrover/data/lens2-aws/ice/static/grid.zarr",
    consolidated=True,
    mode="w",
)

### Land grid

In [48]:
# experiment number
exp_num = 1021

# component
comp = 'lnd'

# variable
var = 'FSNO'

# variable for this dataset
path = col.search(component=comp, variable=var, experiment_number=exp_num).df.path.values[0]

In [50]:
lnd_grid = create_grid_dataset(path, variables=[var])
lnd_grid

<xarray.Dataset>
Dimensions:   (lat: 192, levdcmp: 25, levgrnd: 25, levlak: 10, lon: 288)
Coordinates: (12/18)
  * levgrnd   (levgrnd) float32 0.01 0.04 0.09 0.16 ... 13.33 19.48 28.87 42.0
  * levlak    (levlak) float32 0.05 0.6 2.1 4.6 8.1 12.6 18.6 25.6 34.33 44.78
  * levdcmp   (levdcmp) float32 0.01 0.04 0.09 0.16 ... 13.33 19.48 28.87 42.0
  * lon       (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat       (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    area      (lat, lon) float32 ...
    ...        ...
    WATSAT    (levgrnd, lat, lon) float32 ...
    SUCSAT    (levgrnd, lat, lon) float32 ...
    BSW       (levgrnd, lat, lon) float32 ...
    HKSAT     (levgrnd, lat, lon) float32 ...
    ZLAKE     (levlak, lat, lon) float32 ...
    DZLAKE    (levlak, lat, lon) float32 ...
Data variables:
    *empty*

In [34]:
path = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/SALT/b.e21.BHISTcmip6.f09_g17.LE2-1021.002.pop.h.SALT.187001-187912.nc'
ocean_grid = create_grid_dataset(path, variables=["SALT"])
ocean_grid

<xarray.Dataset>
Dimensions:                 (lat_aux_grid: 395, moc_comp: 3, moc_z: 61, nlat: 384, nlon: 320, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates: (12/64)
    moc_components          (moc_comp) |S384 ...
    transport_components    (transport_comp) |S384 ...
    transport_regions       (transport_reg) |S384 ...
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
    ...                      ...
    hflux_factor            float64 ...
    fwflux_factor           float64 ...
    salinity_factor         float64 ...
    sflux_factor            float64 ...
    nsurface_t              float64 ...
    nsurface_u              float64 ...
Dimensions without coordinates: moc_comp, nlat, nlon, transport_comp, transport_reg
Data variables:
    *empty*

In [35]:
ocean_grid.to_zarr(
    "/glade/scratch/mgrover/data/lens2-aws/ocn/static/grid.zarr",
    consolidated=True,
    mode="w",
)